#### <h1> <center> ENSF 592 Programming Fundamentals for Data Engineers </center></h1>
<h2> <center> Assignment 5: Numpy (5 points)</center></h2>

<center>
<div class="alert alert-block alert-info">
Due: Wed Oct 30 12am (midnight, Tue to Wed). To be submitted on D2L.
</div></center>


Edit this file and write your solutions to the problems in sections specified with `# Your solution goes here`. Test your code and when you are ready, download this notebook as an `.ipynb` file and submit it to D2L. To get this file, in Jupyter notebook you can go to File -> Download as -> Notebook(.ipynb)

# Introduction

### Before You start: 
* Numpy is already installed with the Anaconda stack. You can check with:<br> 
    `import numpy as np
    np.__version__`

* Download data.csv and put it somewhere in your repository to be accessible in you code (for ease of access put it in the same directory of you notebook file)

<div class="alert alert-block alert-info">
<b>Tip:</b> Use `np?` or `np.info(np.<function-to-look-up>)` to read numpy documentation.
</div>


### Description:
We are going to study data collected from 293 patients with heart disease, and extract some meaningfull information. You can download the dataset (data.csv) from d2l and if you want to know more about it, you can use the below Kaggle link:<br>
https://www.kaggle.com/imnikhilanand/heart-attack-prediction



|Feature|Description|
|-------|----------------------------|
|age|age in years|
|gender|(1 = male; 0 = female)|
|cp|chest pain type|
|trestbps|resting blood pressure (in mm Hg on admission to the hospital)|
|chol|serum cholestoral in mg/dl|
|fbs|(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)|
|restecg|resting electrocardiographic results|
|thalach|maximum heart rate achieved|
|exang|exercise induced angina (1 = yes; 0 = no)|
|oldpeak|ST depression induced by exercise relative to rest|
|slope|the slope of the peak exercise ST segment|
|ca|number of major vessels (0-3) colored by flourosopy|
|thal|3 = normal; 6 = fixed defect; 7 = reversable defect|
|num|diagnosis of heart disease (angiographic disease status)|

# 1. Numpy Table (5 pts)
### Part I. Table (3 pts)
In Part II we would like to find answers to the following questions:
* Report1: what is the average age of patients ? 
* Report2: report the average `chol` level of people in intervals of 10 years old ([20,30], [30,40],[40,50],[50,60])
* Report3: report the average `trestbps` in people with `chol` of highest level(the highest 30%) and lowest level(the lowest 30%).
* Report4: report percentage of men and women with positive diagnosis of heart disease(`num=1`). 


As a first step, design a class that is responsible for general processes that you'll need.
* Keep header names in a python list, and store the the data rows in a numpy.ndarray as data attributes
* Implement a `readCSV` method which is responsible for reading header and data from a csv file.
* Implement a `printHead` method that print the header and first 10 rows of the table.
* Implement a `sort` method that sort the table based on the specified `column`. (`default='age'`)
* Implement a `deleteRow` method that gets a `row index` and deletes that row from the table. (`default=last row`) 
* Implement a `deleteCol` method that gets a `column index` and deletes that column from the table. (`default=last column`)
* Implement a `getColumn` method that gets a `column` name and after removing `nan` values from it, returns it as a numpy array. 
* Implement a `select` method that gets a `column` name and a `value`, and after removing `nan` values, searches for records with `column=value` and returns that sub-table. 
* Implement a `rangeSelect` method that gets a `column` name and a `begin` and `end` (that define a range), and after removing `nan` values and sorting the table based on `column`, searchs for records with `begin<column<end` and returns that sub-table.
* Implement a `percentageSelect` method that gets a `column` name, a `perc`(percentage) and `index`, and after removing `nan` values and sorting the table based on `column` from that column: 
    * if index ==0 : returns the __first__ `perc*column.size` sub-table.  
    * if index ==-1 : returns the __last__ `perc*column.size` sub-table. 

<div class="alert alert-block alert-danger">
<b>Note:</b> <br>
Do not change the signature of methods and complete the same class structure as below<br>
`select`, `rangeSelect` and `percentageSelect` should return a new `Table` as their output without changing the current Table.<br>
To remove `nan` values and sort, you must reuse the `getColumn` and `sort` methods that you have implemented before.
</div>

In [2]:
import numpy as np

class Table:
    
    def __init__(self,header=[],data=np.array([])):
        self.header = header
        self.data = data
            
    def readCSV(self,filename:str)->None:
        # skip first row
        self.data = np.genfromtxt(filename, delimiter=',',names=True,dtype=float)
        self.header = list(self.data.dtype.names)

    def printHead(self)->None:
        print(self.header, '\n')
        print(self.data[:10])

    def sort(self,column:str='age')->None:
        self.data = np.sort(self.data, order=column)

    def deleteRow(self,index:int=-1)->None:
        self.data = np.delete(self.data, (index), axis=0)

    def deleteCol(self,column:int=-1)->None:
        self.header = self.header[:column] + self.header[column +1:]
        self.data = self.data[self.header]

    def getColumn(self,column:str='age')->np.array: 
        cols = self.data[column]
        array = cols[~np.isnan(cols)]
        return array

    def select(self,column:str,value:float):
        array = (self.data[column] == value)
        newTable = Table(self.header, self.data[array])
        return newTable
    
    def rangeSelect(self,column:str,begin:int,end:int):
        # get the colum arry without 
        array = self.getColumn(column)
        # sorting the colum
        self.sort(column)
        mask = np.in1d(self.data[column], array)
        array = self.data[mask]
        return Table(self.header, array[np.logical_and(self.getColumn(column) >= begin, self.getColumn(column) < end)])
    
    def percentageSelect(self,column:str,perc:float,index:int):
        self.sort(column)
        array = self.getColumn(column)
        mask = np.in1d(self.data[column], array)
        array = self.data[mask]
        size = int((perc/100)*array.size)
        if index == 0:
            return Table(self.header, array[:size])
        elif index == -1:
            return Table(self.header, array[-size:])

In [5]:
Synopsis = Table()
Synopsis.readCSV("data.csv")
Synopsis.printHead()
# Synopsis.sort(column='trestbps')
# Synopsis.printHead()
# Synopsis.deleteRow(0)
# Synopsis.printHead()
# Synopsis.deleteCol(2)
# Synopsis.printHead()
#Synopsis.getColumn('thal')
#newTable = Synopsis.select('thal', 3)
#newTable.printHead()
subTable = Synopsis.rangeSelect('thal',3,7)
subTable.printHead()
# newTable2 = Synopsis.percentageSelect('chol', 30, -1)
# newTable2.printHead()

['age', 'gender', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'] 

[(28., 1., 2., 130., 132., 0., 2., 185., 0., 0., nan, nan, nan, 0.)
 (29., 1., 2., 120., 243., 0., 0., 160., 0., 0., nan, nan, nan, 0.)
 (29., 1., 2., 140.,  nan, 0., 0., 170., 0., 0., nan, nan, nan, 0.)
 (30., 0., 1., 170., 237., 0., 1., 170., 0., 0., nan, nan,  6., 0.)
 (31., 0., 2., 100., 219., 0., 1., 150., 0., 0., nan, nan, nan, 0.)
 (32., 0., 2., 105., 198., 0., 0., 165., 0., 0., nan, nan, nan, 0.)
 (32., 1., 2., 110., 225., 0., 0., 184., 0., 0., nan, nan, nan, 0.)
 (32., 1., 2., 125., 254., 0., 0., 155., 0., 0., nan, nan, nan, 0.)
 (33., 1., 3., 120., 298., 0., 0., 185., 0., 0., nan, nan, nan, 0.)
 (34., 0., 2., 130., 161., 0., 0., 190., 0., 0., nan, nan, nan, 0.)]
['age', 'gender', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'] 

[(36., 1., 3., 112., 340., 0., 0., 184., 0., 1.,  2., nan, 3., 0.)
 (

### Part II. Extract Meaning (2 pts)

Now that you have a proper infrastructure to handle the data, we would like to answer the following questions:
* Report1: what is the average age of patients ? 
* Report2: report the average `chol` level of people in intervals of 10 years old ([20,30], [30,40],[40,50],[50,60])
* Report3: report the average `trestbps` in people with `chol` of highest level(the highest 30%) and lowest level(the lowest 30%).
* Report4: report percentage of men and women with positive diagnosis of heart disease(`num=1`). 

We are expecting the following output:
```
Report1
 	the average age of patients are:  47.767918088737204

Report2
	for people with age in range of 20 30 average CHOL is:  187.5
	for people with age in range of 30 40 average CHOL is:  239.6304347826087
	for people with age in range of 40 50 average CHOL is:  250.13131313131314
	for people with age in range of 50 60 average CHOL is:  256.2105263157895

Report3
	for patients with lowest 30% of chol, average trestbps is:  132.97530864197532
	for patients with highest 30% of chol, average trestbps is:  136.05

Report4
	88.57% of patients diagnosed with heart decease are men and  11.43% of them are women
```


In [232]:
# Your Code goes here - complete the code bellow

report=Table()
report.readCSV("data.csv")

# report 1
print("Report1\n \tthe average age of patients are: ",np.nanmean(report.getColumn('age')))

# report2
print("\nReport2")
for i in range(20,60,10):
    age = report.rangeSelect('age', i, i+10)
    print("\tfor people with age in range of", i,i+10, "average CHOL is: ", np.nanmean(age.getColumn('chol')))

# report3
print("\nReport3")
lowest = report.percentageSelect('chol', 30, 0)
higest = report.percentageSelect('chol', 30, -1)
print("\tfor patients with lowest 30% of chol, average trestbps is: ",np.nanmean(lowest.getColumn('trestbps')))
print("\tfor patients with highest 30% of chol, average trestbps is: ",np.nanmean(higest.getColumn('trestbps')))

# report4
print("\nReport4")
positiveDisease = report.select('num', 1)
positiveDisease.getColumn('gender')
numOfMen = 0
numOfWomen = 0
for i in positiveDisease.getColumn('gender'):
    if i == 0:
        numOfWomen += 1
    elif i == 1:
        numOfMen += 1
percNumOfMen = (numOfMen/len(positiveDisease.getColumn('gender')))*100
percNumOfWomen = (numOfWomen/len(positiveDisease.getColumn('gender')))*100
print("\t{0:.2f}% of patients diagnosed with heart decease are men and ".format(percNumOfMen), "{0:.2f}% of them are women".format(percNumOfWomen))

Report1
 	the average age of patients are:  47.767918088737204

Report2
	for people with age in range of 20 30 average CHOL is:  187.5
	for people with age in range of 30 40 average CHOL is:  239.6304347826087
	for people with age in range of 40 50 average CHOL is:  250.13131313131314
	for people with age in range of 50 60 average CHOL is:  256.2105263157895

Report3
	for patients with lowest 30% of chol, average trestbps is:  132.97530864197532
	for patients with highest 30% of chol, average trestbps is:  135.6375

Report4
	88.57% of patients diagnosed with heart decease are men and  11.43% of them are women


# 2. Numpy - remove duplicate rows (optional)

If rows contain numerical data, finding duplicates might mean finding rows that are close in a distance sense, rather than exactly the same. Each column would be treated as a dimension, and the multi-dimensional distance lower than a threshold would mean rows are equal.

In this stackexchange thread https://stackoverflow.com/questions/16970982/find-unique-rows-in-numpy-array

the following solution is presented to implement this:
```python
from scipy.spatial.distance import squareform, pdist

def unique_rows(arr, thresh=0.0, metric='euclidean'):
    "Returns subset of rows that are unique, in terms of Euclidean distance"
    distances = squareform(pdist(arr, metric=metric))
    idxset = {tuple(np.nonzero(v)[0]) for v in distances <= thresh}
    return arr[[x[0] for x in idxset]]

```

Study this method by running each step separately and explain. Create test arrays to show that it works as intended. Does it always work?


In [ ]:
# Your solution goes here

# 3. Statistics for Hackers (optional)

Watch Jake Vanderplas' talk Statistics for Hackers - PyCon 2016 https://www.youtube.com/watch?v=Iq9DzN6mvYA

Using the described _shuffeling_ technique and the heart attack dataset, investigate if _young_ individuals have lower blood pressure than _old_ individuals and if this is statistically significant. 

Note that `np.random.shuffle()` can shuffle arrays. For an example see: https://github.com/croach/statistics-for-hackers


In [ ]:
# Your solution goes here